In [1]:

from pytesis.intervals import *
from pytesis.power import *
from pytesis.datasets import arc, eyeglasses, filled_circle, add_noise, add_outliers
from pytesis.datasets import plot_dataset
from pytesis.fermat import fermat_dist
from sklearn.metrics import pairwise_distances
from functools import partial
import numpy as np
import pandas as pd

from functools import partial



In [2]:
B = 10
dataset_factory = partial(eyeglasses, n=1000, bridge_height=0.5, r=1, separation=3)
result = distance_power(
    dataset_factory,
    band=0.1,
    dist_function=None,
    B=B,
    ncores=1
)

INFO:power:Iteration: 0
INFO:power:Iteration: 1
INFO:power:Iteration: 2
INFO:power:Iteration: 3
INFO:power:Iteration: 4
INFO:power:Iteration: 5
INFO:power:Iteration: 6
INFO:power:Iteration: 7
INFO:power:Iteration: 8
INFO:power:Iteration: 9


In [4]:
result.table

,holes,counts,percentage
0,2,10,1.0
